In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext('local[*]')

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [17]:
from collections import namedtuple

Person = namedtuple('employee',['name','age','address'])
Address = namedtuple('address', ['city','country'])

row1 = Person('Bob', 35, Address('London','UK'))
row2 = Person('Susan', 42, Address('Amsterdam','NL'))
row3 = Person('Sara', 29, Address('Boulder','USA'))

people = sqlContext.createDataFrame([row1, row2, row3])


In [18]:
people.where(people['age']>30).show()

+-----+---+--------------+
| name|age|       address|
+-----+---+--------------+
|  Bob| 35|   [London,UK]|
|Susan| 42|[Amsterdam,NL]|
+-----+---+--------------+



In [27]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import ParseError
from pyspark.sql import Row

def parse_xml_to_dict(xmlString):
    data = {'title':None, 'redirect_title':None, 'timestamp':None, "last_contributor_username": None, "text":None}
    
    try:
        root = ET.fromstring(xmlString.encode('utf-8'))
        title = root.find('title')
        if title is not None:
            data['title'] = title.text
        redirect = root.find('redirect')
        if redirect is not None:
            data['redirect_title'] = redirect.attrib['title'] 
        revision = root.find("revision")
        if revision is not None:
            timestamp = revision.find("timestamp")
            data["timestamp"] = timestamp.text
        contributor = revision.find("contributor")
        if contributor is not None:
            username = contributor.find("username")
        if username is not None:
            data["last_contributor_username"] = username.text
        text = revision.find("text")
        
        if text is not None and text.text is not None:
            data["text"] = text.text.replace("\\n", " ")
    except ParseError:
        data['title'] = '<PARSE ERROR>'

    return data #Row(**dict)

In [28]:
import codecs
import json
from xml.etree.ElementTree import ParseError

wikiData = codecs.open('/Users/linamiao/Downloads/enwiki-20170320-stub-articles1.xml', 'r', 'utf-8')
jsonData = codecs.open('/Users/linamiao/Downloads/allpages.json', 'w', 'utf-8')

pageData = []
articleCount = 0
pageCount = 0
pageStart = 0

for i, line in enumerate(wikiData):
    #if i > 10000:
    #    break

    if '<page>' in line:
        pageCount += 1

        if pageCount > 1:
            print 'unexpected to have pageCount > 1'
        else:
            articleCount += 1
            pageStart = line.index('<page>')

    if pageCount > 0:
        pageData.append(line[pageStart:])

    if '</page>' in line:
        pageCount -= 1

        if pageCount == 0:
            try:
                fromxml = parse_xml_to_dict(u'\n'.join(pageData))
            except ParseError:
                print u'\n'.join(pageData)
                break

            json.dump(fromxml, jsonData)
            jsonData.write('\n')
            pageData = []

jsonData.close()
wikiData.close()

In [29]:
df = sqlContext.read.json("/Users/linamiao/Downloads/allpages.json")
df.write.parquet("/Users/linamiao/Downloads/allpages.parquet")
df2 = sqlContext.read.parquet("/Users/linamiao/Downloads/allpages.parquet")
dfOne = df2.sample(False, .01, 2718).coalesce(24)
dfOne.write.parquet('/Users/linamiao/Downloads/onepercent.parquet')
dfSmall = df2.sample(False, .0005, 2718).coalesce(8)
dfSmall.write.parquet("/Users/linamiao/Downloads/smallwiki.parquet")